In [11]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation


%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [12]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [1000]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

In [13]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

(7521, 4)
(7521, 4)
(7521, 2)
(89, 64)


In [24]:
nb_inputs  = 8
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 128

import scipy.io

from dataset import Dataset, SpikingDataset, ARegDataset, RegSpikingDataset, RegTorchBothDataset
train_data = RegTorchBothDataset(train_input, train_label, nb_inputs, nb_steps)
test_data = RegTorchBothDataset(test_input, train_label, nb_inputs, nb_steps)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
print(len(snn_train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)
print(train_data[0][2].shape)
print(train_data[0][3].shape)

7521
(32, 100)
(1,)
(8, 4)
(2,)


In [25]:



class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : 1.00,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,     
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #slayer.block.cuba.Input(neuron_params),
                slayer.block.cuba.Recurrent(neuron_params, input_size, 64, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 32, 64, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 64, 128, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 128, output_size, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, input_size, 64, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 64, 128, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 128, output_size, weight_scale=2, weight_norm=True)
                #slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                #slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5)
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

class Decoder(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16 + 1, 8)
        self.fc3 = nn.Linear(8, 2)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x, x1):
        x = x.flatten(start_dim=1)
        #x = torch.cat((x, x1), axis=1)
        x = self.fc1(x)
        x = self.act(x) 
        x = torch.cat((x, x1), axis=1)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        #x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc3(x)
        #x = self.act(x)
        #x = self.fc4(x)
        return x

# DNN network 

In [48]:
rnn = nn.LSTM(10, 20, 2)

class LSTMNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x):
        x, _ = self.lstm(x)
        x = self.fc1(x[:, -1, :])
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        return x

In [49]:
net_snn = Network(nb_inputs * 4, 2).to(device)
decoder = Decoder(2 * 100, 2).to(device)

net_lstm = LSTMNet(4, 2).to(device)

for name, weight in net_snn.named_parameters():
    print(name, weight.shape)


blocks.0.neuron.current_decay torch.Size([1])
blocks.0.neuron.voltage_decay torch.Size([1])
blocks.0.delay.delay torch.Size([1])
blocks.0.input_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.input_synapse.weight_v torch.Size([64, 32, 1, 1, 1])
blocks.0.recurrent_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.recurrent_synapse.weight_v torch.Size([64, 64, 1, 1, 1])
blocks.1.neuron.current_decay torch.Size([1])
blocks.1.neuron.voltage_decay torch.Size([1])
blocks.1.delay.delay torch.Size([1])
blocks.1.synapse.weight_g torch.Size([128, 1, 1, 1, 1])
blocks.1.synapse.weight_v torch.Size([128, 64, 1, 1, 1])
blocks.2.neuron.current_decay torch.Size([1])
blocks.2.neuron.voltage_decay torch.Size([1])
blocks.2.synapse.weight_g torch.Size([2, 1, 1, 1, 1])
blocks.2.synapse.weight_v torch.Size([2, 128, 1, 1, 1])


In [56]:
def mutual_loss(teach_output, student_output, alpha, target):
    # student loss 1
    loss1 = torch.mean((student_output - target)**2)
    loss2 = torch.mean((student_output - teach_output)**2)
    loss = (1 - alpha) * loss1 + alpha * loss2
    return loss

def all_snn_model(x1, x2, model, decoder):
    output = model(x1)
    output = decoder(output, x2)
    return output
    
    

def mutual_train(trainloader, testloader, snn_model, decoder, dnn_model, lr=2e-3, nb_epochs=10):
    params = list(snn_model.parameters()) + list(decoder.parameters())
    optimizer_snn = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    optimizer_dnn = torch.optim.Adam(dnn_model.parameters(), lr=lr, betas=(0.9,0.999))
    scheduler_snn = torch.optim.lr_scheduler.StepLR(optimizer_snn, step_size=100, gamma=0.1)
    scheduler_dnn = torch.optim.lr_scheduler.StepLR(optimizer_dnn, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    
    for e in range(nb_epochs):
        print(e)
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_snn.zero_grad()
            optimizer_dnn.zero_grad()
            snn_output = all_snn_model(x1_local, x2_local, snn_model, decoder)
            dnn_output = dnn_model(x3_local)
            
            snn_loss = mutual_loss(dnn_output.detach(), snn_output, alpha=0.1, target=y_local)
            dnn_loss = mutual_loss(snn_output.detach(), dnn_output, alpha=0.1, target=y_local)
            snn_loss.backward()
            dnn_loss.backward()
            optimizer_snn.step()
            optimizer_dnn.step()
            loss_hist.append(snn_loss.item() + dnn_loss.item())
        print(np.mean(loss_hist))
        
         
        

In [57]:
mutual_train(train_loader, train_loader, net_snn, decoder, net_lstm, lr=1e-3, nb_epochs=100)

0
0.043825347075175686
1
0.037636231319284286
2
0.03553285400414745
3
0.03439109357689523
4
0.034266040853808744
5
0.034082308541825514
6
0.03326823959379631
7
0.03291601019564195
8
0.03099907861462147
9
0.029383155033860544
10
0.027876510107258366
11
0.02620818137025492
12
0.02474479164367826
13
0.019524440163414987
14
0.016941420455715794
15
0.01590654580119871
16
0.013085434734189914
17
0.012196117163620006
18
0.011239294569457929
19
0.01031244092253905
20
0.009854796091962796
21
0.009428580308834218
22
0.0087826808513008
23
0.008212285792608982
24
0.007253291370974601
25
0.007342300538597155
26
0.006702015318481599
27
0.006520603996034617
28
0.006182447214950255
29
0.006652373635068031
30
0.005603012968726866
31
0.0050537749677028315
32
0.005157093721702379
33
0.005573950290593113
34
0.005143548276125438
35
0.005643831910782541
36
0.005448287727816378
37
0.005685806107683644
38
0.005375974968478273
39
0.005258994118282887
40
0.005170077965515903
41
0.004861415624302827
42
0.0048106

In [28]:
net_snn.export_hdf5('./net_snn.net')
torch.save(net_dnn.state_dict(), './net_dnn.pth')

In [21]:
error = 0.0
inputs = []
labels = []
loss_fn = torch.nn.MSELoss()
all_loss =[]
test_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=True)
for input, input1, target in test_loader:
    inputs.append(input)
    labels.append(target)
    target = torch.tensor(target).float()
    input = torch.tensor(input).float()
    input1 = torch.tensor(input1).float()
    
    output = net_snn(input)
    output = net_dnn(output, input1)
    print(output.shape)
    #print(output, target)
    loss_val = loss_fn(output, target)
    all_loss.append(loss_val.item())
print(np.mean(all_loss))
inputs = np.array(inputs)
labels = np.array(labels)
#inputs = inputs.reshape((inputs.shape[0], -1))
print(inputs.shape)
rc.my_test()
np.save('input_snn.npy', inputs)
np.save('label_snn.npy', labels)

/tmp/ipykernel_77/3015334240.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_77/3015334240.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_77/3015334240.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()


torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])
torch.Size([128, 2])


KeyboardInterrupt: 

In [59]:
all_output = []
inputs = []
labels = []

test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
for input, input1, _, target in test_loader:
    inputs.append(input)
    labels.append(target)
    target = torch.tensor(target).float()
    input = torch.tensor(input).float()
    input1 = torch.tensor(input1).float()
    
    output = net_snn(input)
    output = decoder(output, input1).cpu().detach().numpy()
    all_output.append(output)
    
all_output = np.concatenate(all_output, axis=0)
ber = rc.my_test(all_output)
print(ber)

/tmp/ipykernel_788/494841858.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_788/494841858.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_788/494841858.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()


0.17778558052434457


In [11]:
A1 = 8 * torch.ones((128, 8))
A2 = torch.ones((128, 1))
print(A1 + A2)

tensor([[9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        ...,
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.]])
